# Exploratory Model Analysis

*Purpose*: **TODO**.


## Setup


In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

In [ ]:
from grama.models import make_prlc_rand
md_circuit = make_prlc_rand()

# Basic Facts


### __qX__ Model summary


*Observations*

- ...
  - (Your response here)


## Model context

The deterministic variables `L, R, C` are the *designed* component values.

The random variables `dL, dR, dC` are *perturbations* to the designed component values.


# Inputs

## Assess input variability


### __qX__ Overview of inputs


In [ ]:
(
    md_circuit

)

*Observations*

- ...
  - (Your response here)


### __qX__ Compare designed and realized values

The following plot shows the designed and realized capacitance values. Inspect the plot, then change to log scales for both the x and y axes. Answer the questions under *observations* below.


In [ ]:
# TASK: Inspect the following plot, change to a log scale
# for both x and y
(
    # NOTE: No need to edit this part of the code
    md_circuit
    >> gr.ev_sample(
        n=20,
        df_det=gr.df_make(
            R=1e-1,
            L=1e-5,
            C=gr.logspace(-3, 2, 20)
        )
    )
    # Visualize
    >> gr.ggplot(gr.aes("C", "Cr"))
    + gr.geom_point()
    # TASK: Add a log scale for x and y

)

*Observations*

- ...
  - (Your response here)


# Outputs


### __qX__ Overview of outputs


In [ ]:
(
    md_circuit

)

*Observations*

*Note*: If you can't reliably make out the shapes of the distributions, try increasing `n`.

- What distribution shapes do the realized component values `Lr, Rr, Cr` have?
  - (Your response here)
- What distribution shape does the output `Q` have?
  - (Your response here)
- What distribution shape does the output `omega0` have?
  - (Your response here)


### __qX__ Scatter of outputs


In [ ]:
(
    md_circuit
    >> gr.ev_sample(n=1e3, df_det="nom")

)


*Observations*

- ...
  - (Your response here)


### __qX__ Density of outputs

Visualize `Q` and `omega0` with a 2d bin plot. Increase the sample size `n` to get a "full" plot.


In [ ]:
(
    md_circuit

)

*Observations*

- Briefly describe the distribution of realized performance (values of `Q` and `omega0`).
  - (Your response here)


### __qX__ Compare nominal with distribution


In [ ]:
(
    md_circuit
    >> gr.ev_sample(n=1e4, df_det="nom")
    
    >> gr.ggplot(gr.aes("Q", "omega0"))
    + gr.geom_bin2d()
    + gr.geom_point(
        # Use the `data` argument to add a geometry for
        # the output value associated with nominal input values

        color="salmon",
        size=4,
    )
)

*Observations*

The nominal design (red point) represents the predicted performance if we assume the nominal circuit component values. 

- How does the distribution of real circuit performance (values of `Q`, `omega0`) compare with the nominal performance?
  - (Your response here)
- Is the most likely performance (values of `Q`, `omega0`) the same as the nominal performance? ("Most likely" is where `count` is the largest.)
  - (Your response here)
- Assume that another system depends on the particular values of `Q` and `omega0` provided by this system. Would it be safe to assume the performance is within 1% of the nominal performance?
  - (Your response here)


# Input-to-output Relationships


### __qX__ Correlation tile plot



In [ ]:
(
    md_circuit
    >> gr.ev_sample(
        n=1e3, 
        df_det="nom",
    )

    >> gr.pt_auto()
)

*Observations*

- ...
  - (Your response here)


### __qX__ Targeted scatterplots


### __qX__ Sinew plot


In [ ]:
(
    md_circuit

)

*Observations*

- ...
  - (Your response here)
